In [7]:
from gurobipy import *
import pandas as pd
import numpy as np

In [19]:
# arrange origin and market with code
df_geo = pd.read_csv('data/geo_data.csv')
df_s2m = pd.read_excel("data/s2m.xlsx")
df_market = pd.DataFrame({"code": range(df_s2m.shape[0]), "name": df_s2m["Unnamed: 0"]})
df_market = df_market.merge(right = df_geo, left_on = "name", right_on = "place")[["name", "code", "lat", "lng"]]
df_market["code"] += 1
df_market.to_csv('data/market_code.csv')
display(df_market)
# 建立縣市代碼
name = [i for i in df_s2m.columns if i != "Unnamed: 0"]
df_origin = pd.DataFrame({"code": range(df_s2m.shape[1]-1), "name": name})
df_origin = df_origin.merge(right = df_geo, left_on = "name", right_on = "place")[["name", "code", "lat", "lng"]]
df_origin.to_csv('data/county_code.csv')
display(df_origin)
# df_s2m

,name,code,lat,lng
0,台北二綜合農產品批發市場,1,25.017152,121.497170
1,台北一綜合農產品批發市場,2,25.017135,121.497064
2,三重區綜合農產品批發市場,3,25.065173,121.486490
3,桃農綜合農產品批發市場,4,24.990382,121.269176
4,台中市綜合農產品批發市場,5,24.199446,120.658159
5,永靖鄉綜合農產品批發市場,6,23.922792,120.550124
6,溪湖鎮綜合農產品批發市場,7,23.957270,120.486067
7,南投市綜合農產品批發市場,8,24.990382,121.269176
8,西螺鎮綜合農產品批發市場,9,23.796092,120.458872
9,高雄市綜合農產品批發市場,10,22.644275,120.315982


,name,code,lat,lng
0,新北市,0,25.016983,121.462787
1,台北市,1,25.032964,121.565427
2,桃園市,2,24.991703,121.298959
3,台中市,3,24.147736,120.673648
4,台南市,4,22.999900,120.226876
5,高雄市,5,22.627278,120.301435
6,宜蘭縣,6,24.702107,121.737750
7,新竹縣,7,24.838723,121.017725
8,苗栗縣,8,24.560159,120.821427
9,彰化縣,9,24.051796,120.516135


In [16]:
df_origin

,code,name,Unnamed: 0,place,lat,lng
0,0,新北市,0,新北市,25.016983,121.462787
1,1,台北市,1,台北市,25.032964,121.565427
2,2,桃園市,2,桃園市,24.991703,121.298959
3,3,台中市,3,台中市,24.147736,120.673648
4,4,台南市,4,台南市,22.999900,120.226876
5,5,高雄市,5,高雄市,22.627278,120.301435
6,6,宜蘭縣,6,宜蘭縣,24.702107,121.737750
7,7,新竹縣,7,新竹縣,24.838723,121.017725
8,8,苗栗縣,8,苗栗縣,24.560159,120.821427
9,9,彰化縣,9,彰化縣,24.051796,120.516135


In [9]:
J = range(df_market.shape[0])

In [10]:
# quantity單位：公噸
# 2013 - 2021
df_production = pd.read_csv("data/production.csv")
df_production = df_production[df_production["region"] != "金門縣"]
df_production = df_production[df_production["region"] != "澎湖縣"]
df_production = df_production[df_production["region"] != "連江縣"]
df_production["quantity"] = pd.to_numeric(df_production["quantity"])
df_production = df_production[["region", "quantity"]].groupby(['region']).mean()
df_production = pd.DataFrame({"name": df_production.quantity.index, "quantity": df_production.quantity.values})
# 公噸轉公斤
df_production['quantity'] = pd.to_numeric(df_production['quantity']) * 1000
df_production = df_production.merge(right = df_origin, on = "name")
df_production.sort_values(by = "code", ignore_index = True, inplace = True)
df_production.to_csv('data/county_production.csv')
df_production

,name,quantity,code
0,新北市,3.073249e+06,0
1,台北市,5.496999e+05,1
2,桃園市,3.206673e+06,2
3,台中市,3.527415e+07,3
4,台南市,1.918369e+07,4
5,高雄市,7.677425e+06,5
6,宜蘭縣,8.633788e+07,6
7,新竹縣,7.427260e+06,7
8,苗栗縣,4.604948e+06,8
9,彰化縣,4.531875e+07,9


In [11]:
df_demand = pd.read_excel("data/demand_function.xlsx")
df_demand

,Unnamed: 0,ID,slope,intercept
0,台北二,104,-0.00006,30.275
1,台北一,109,-0.00003,30.715
2,三重區,241,-0.00020,27.579
3,桃農,338,-0.00010,26.032
4,台中市,400,-0.00020,26.975
5,永靖鄉,512,-0.00020,22.064
6,溪湖鎮,514,-0.00010,22.777
7,南投市場,540,-0.00440,28.924
8,西螺鎮,648,-0.00003,29.549
9,高雄市,800,-0.00010,29.729


In [12]:
# consumption
df_consumption = pd.read_csv("data/consumption.csv")
df_consumption = df_consumption[df_consumption["market_name"].isin(list(df_demand["Unnamed: 0"]))]
df_consumption["month"] = df_consumption["date"].str.split('.').str[1]
consumptions = []
month = df_consumption["month"].unique()
for i in month:
    consumptions.append(df_consumption[df_consumption["month"] == i][["market_name", 
                        "quantity", "mean_price"]].groupby(["market_name"]).mean())
print(consumptions)


[                  quantity  mean_price
market_name                           
三重區           50967.553936   12.080210
台中市           42682.312865   11.989855
台北一          378746.411079   13.322492
台北二          157183.655977   13.280257
台東市           14901.787582   15.333606
宜蘭市           20436.245353    9.801510
屏東市           33785.737609   12.140778
板橋區           88827.083436   12.826775
桃農            32963.338192   12.052526
永靖鄉           56868.415441    9.967664
溪湖鎮           84505.971098   12.446673
花蓮市           25497.405917   15.766917
西螺鎮          379836.636637   11.044560
豐原區           40815.417040   10.716375
高雄市           79209.460641   15.881483
鳳山區           24236.137026   11.331352,                   quantity  mean_price
market_name                           
三重區           48038.578397   10.238284
台中市           40475.560976   10.075852
台北一          361029.118467   11.116382
台北二          150816.038328   11.141501
台東市           14696.840637   13.780513
宜蘭市           15588.349

In [13]:
export consumptions

SyntaxError: invalid syntax (4127550129.py, line 1)

In [14]:
df_geo = pd.read_csv('data/geo_data.csv')
df_origin = df_.merge(right = df_geo, left_on = "name", right_on = "place")
df_county

NameError: name 'df_county' is not defined